In [9]:
import requests
from datetime import datetime
import pandas as pd
from html import unescape
from bs4 import BeautifulSoup
import unicodedata


In [10]:
url_base = "https://www.bcb.gov.br/api/servico/sitebcb/copom"
atas_resource = "/atas"
detail_ata_resource = "/atas_detalhes"
quantity = 500


In [11]:
dataset = []

In [12]:
def get_atas():
    try:
        response = requests.get(url_base+atas_resource+'?quantidade='+str(quantity))
        return response
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)    

In [13]:
def get_ata(nro_reuniao):
    try:
        response = requests.get(url_base + detail_ata_resource +'?nro_reuniao='+str(nro_reuniao))
        return response
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)


In [14]:
reunioes = get_atas().json()
print(reunioes)

{'conteudo': [{'nroReuniao': 269, 'dataReferencia': '2025-03-19', 'dataPublicacao': '2025-03-25', 'titulo': '269ª Reunião - 18-19 março, 2025'}, {'nroReuniao': 268, 'dataReferencia': '2025-01-29', 'dataPublicacao': '2025-02-04', 'titulo': '268ª Reunião - 28-29 janeiro, 2025'}, {'nroReuniao': 267, 'dataReferencia': '2024-12-11', 'dataPublicacao': '2024-12-17', 'titulo': '267ª Reunião - 10-11 dezembro, 2024'}, {'nroReuniao': 266, 'dataReferencia': '2024-11-06', 'dataPublicacao': '2024-11-14', 'titulo': '266ª Reunião - 5-6 novembro, 2024'}, {'nroReuniao': 265, 'dataReferencia': '2024-09-18', 'dataPublicacao': '2024-09-24', 'titulo': '265ª Reunião - 17-18 setembro, 2024'}, {'nroReuniao': 264, 'dataReferencia': '2024-07-31', 'dataPublicacao': '2024-08-06', 'titulo': '264ª Reunião - 30-31 julho, 2024'}, {'nroReuniao': 263, 'dataReferencia': '2024-06-19', 'dataPublicacao': '2024-06-25', 'titulo': '263ª Reunião - 18-19 junho, 2024'}, {'nroReuniao': 262, 'dataReferencia': '2024-05-08', 'dataPub

In [15]:
for nreuniao in reunioes["conteudo"]:
    nro = nreuniao["nroReuniao"]
    data_reuniao = datetime.strptime(nreuniao["dataReferencia"], "%Y-%m-%d").strftime("%Y%m%d")
    print(f"N: {nro}. Data: {data_reuniao}")

    detalhes = get_ata(nro).json()
    if detalhes["conteudo"]:
        texto_html = detalhes["conteudo"][0]["textoAta"]
        if texto_html:
            texto = unescape(texto_html)
            soup = BeautifulSoup(texto, "html.parser")
            texto_limpo = soup.get_text(separator=" ", strip=True)
        else:
            texto_limpo = ""
            print(f"Nenhum conteúdo encontrado para a reunião {nro}")

        dataset.append({
            "data": int(data_reuniao),
            "texto_ata": texto_limpo,
            "pdf": 0
        })
    else:
        print(f"Nenhum conteúdo encontrado para a reunião {nro}")

    

N: 269. Data: 20250319
N: 268. Data: 20250129
N: 267. Data: 20241211
N: 266. Data: 20241106
N: 265. Data: 20240918
N: 264. Data: 20240731
N: 263. Data: 20240619
N: 262. Data: 20240508
N: 261. Data: 20240320
N: 260. Data: 20240131
N: 259. Data: 20231213
N: 258. Data: 20231101
N: 257. Data: 20230920
N: 256. Data: 20230802
N: 255. Data: 20230621
N: 254. Data: 20230503
N: 253. Data: 20230322
N: 252. Data: 20230201
N: 251. Data: 20221207
N: 250. Data: 20221026
N: 249. Data: 20220921
N: 248. Data: 20220803
N: 247. Data: 20220615
N: 246. Data: 20220504
N: 245. Data: 20220316
N: 244. Data: 20220202
N: 243. Data: 20211208
N: 242. Data: 20211027
N: 241. Data: 20210922
N: 240. Data: 20210804
N: 239. Data: 20210616
N: 238. Data: 20210505
N: 237. Data: 20210317
N: 236. Data: 20210120
N: 235. Data: 20201209
N: 234. Data: 20201028
N: 233. Data: 20200916
N: 232. Data: 20200805
N: 231. Data: 20200617
Nenhum conteúdo encontrado para a reunião 231
N: 230. Data: 20200506
Nenhum conteúdo encontrado para a 

In [16]:
df_pt = pd.DataFrame(dataset)

In [17]:
df_pt['decisao'] = None
df_pt

,data,texto_ata,pdf,decisao
0,20250319,A) Atualização da conjuntura econômica e do ce...,0,None
1,20250129,A) Atualização da conjuntura econômica e do ce...,0,None
2,20241211,A) Atualização da conjuntura econômica e do ce...,0,None
3,20241106,A) Atualização da conjuntura econômica e do ce...,0,None
4,20240918,A) Atualização da conjuntura econômica e do ce...,0,None
...,...,...,...,...
244,19980624,Sumário Preços e Nível de Atividade Agregados ...,0,None
245,19980520,Sumário Preços e Nível de Atividade Agregados ...,0,None
246,19980415,Sumário Preços e Nível de Atividade Agregados ...,0,None
247,19980304,Sumário Preços e Nível de Atividade Agregados ...,0,None


In [18]:
print(df_pt[df_pt['data'].astype(int) < 20000101])

         data                                          texto_ata  pdf decisao
227  19991215  Sumário Demanda e oferta agregadas Preços Ambi...    0    None
228  19991110  Sumário Demanda e oferta agregadas Ambiente Ex...    0    None
229  19991006  Sumário Demanda e oferta agregadas Ambiente Ex...    0    None
230  19990922  Sumário Demanda e oferta agregadas Ambiente ex...    0    None
231  19990901  Sumário Demanda e oferta agregadas Preços Agre...    0    None
232  19990728  Sumário Preços e nível de atividade Agregados ...    0    None
233  19990623  Sumário Preços e Nível de Atividade Agregados ...    0    None
234  19990519  Sumário Preços e Nível de Atividade Agregados ...    0    None
235  19990414  Sumário Preços e Nível de Atividade Agregados ...    0    None
236  19990304  Sumário Preços e Nível de Atividade Agregados ...    0    None
237  19990118  Sumário Preços e Nível de Atividade Balanço de...    0    None
238  19981216  Sumário Preços e Nível de Atividade Agregados ...

In [19]:
df = df_pt.drop(df_pt[df_pt['data'].astype(int)<20000101].index)
df

,data,texto_ata,pdf,decisao
0,20250319,A) Atualização da conjuntura econômica e do ce...,0,None
1,20250129,A) Atualização da conjuntura econômica e do ce...,0,None
2,20241211,A) Atualização da conjuntura econômica e do ce...,0,None
3,20241106,A) Atualização da conjuntura econômica e do ce...,0,None
4,20240918,A) Atualização da conjuntura econômica e do ce...,0,None
...,...,...,...,...
222,20000524,Sumário Demanda e oferta agregadas Ambiente ex...,0,None
223,20000419,Sumário Demanda e oferta agregadas Ambiente ex...,0,None
224,20000322,Sumário Demanda e oferta agregadas Ambiente ex...,0,None
225,20000216,Sumário Demanda e oferta agregadas Ambiente ex...,0,None


In [20]:
#Completar dataset com textos faltantes
#print(df_pt[(df_pt['texto_ata'] == '') | (df_pt['texto_ata'].isna())].values)
df_faltantes = df_pt[(df_pt['texto_ata'] == '') | (df_pt['texto_ata'].isna())]
print(df_faltantes)

print(len(df_faltantes))


        data texto_ata  pdf decisao
38  20200617              0    None
39  20200506              0    None
40  20200318              0    None
41  20200205              0    None
42  20191211              0    None
43  20191030              0    None
44  20190918              0    None
45  20190731              0    None
46  20190619              0    None
47  20190508              0    None
48  20190320              0    None
49  20190206              0    None
50  20181212              0    None
51  20181031              0    None
52  20180919              0    None
53  20180801              0    None
54  20180620              0    None
55  20180516              0    None
56  20180321              0    None
57  20180207              0    None
58  20171206              0    None
59  20171025              0    None
60  20170906              0    None
61  20170726              0    None
62  20170531              0    None
63  20170412              0    None
64  20170222              0 

In [21]:
import os
import re
import fitz
import pandas as pd


PDF_DIR = '../../dataset/pdf_pt'
textos_complementares = []

In [ ]:
for arquivo in os.listdir(PDF_DIR):
    if arquivo.lower().endswith(".pdf"):
        caminho_pdf = os.path.join(PDF_DIR, arquivo)

        try:
            print(f"Lendo {arquivo}...")
            doc = fitz.open(caminho_pdf)
            texto = ""
            for pagina in doc:
                texto += pagina.get_text()
            doc.close()

            # Extrair a data do nome do arquivo
            data_match = re.search(r'(\d{8})', arquivo)
            data_pdf = data_match.group(1) if data_match else "data_desconhecida"

            df_faltantes['data'] = df_faltantes['data'].astype(str)

            print(data_pdf)

            if data_pdf in df_faltantes['data'].values:
                print(f"Data encontrada nos textos faltantes {data_pdf}")

                textos_complementares.append({
                    "data": data_pdf,
                    "texto_ata": texto.strip(),
                    "pdf": 1,
                    "nome_arquivo": arquivo
                })
            else:
                print(f"Data não encontrada no df_pt {data_pdf}")

        except Exception as e:
            print(f"Erro ao processar {arquivo}: {e}")

df_complementar = pd.DataFrame(textos_complementares)

In [23]:
df_complementar

,data,texto_ata,pdf,nome_arquivo
0,20160720,Notas da 200ª Reunião do \nComitê de Política...,1,COPOM200-not20160720200.pdf
1,20160831,Notas da 201ª Reunião do \nComitê de Política...,1,COPOM201-not20160831201.pdf
2,20161019,Notas da 202ª Reunião do \nComitê de Política...,1,COPOM202-not20161019202.pdf
3,20161130,Notas da 203ª Reunião do \nComitê de Política...,1,COPOM203-not20161130203.pdf
4,20170111,Notas da 204ª Reunião do \nComitê de Política...,1,COPOM204-not20170111204.pdf
5,20170222,Notas da 205ª Reunião do \nComitê de Política...,1,COPOM205-not20170222205.pdf
6,20170412,Notas da 206ª Reunião do \nComitê de Política...,1,COPOM206-not20170412206.pdf
7,20170531,Notas da 207ª Reunião do \nComitê de Política...,1,COPOM207-not20170531207.pdf
8,20170726,Notas da 208ª Reunião do \nComitê de Política...,1,COPOM208-not20170726208.pdf
9,20170906,Notas da 209ª Reunião do \nComitê de Política...,1,COPOM209-not20170906209.pdf


In [24]:
for i, row in df_complementar.iterrows():
    data = row['data']
    texto = row['texto_ata']
    
    # Atualiza o texto e seta pdf=1 onde a data bate
    df_pt.loc[df_pt['data'] == int(data), 'texto_ata'] = texto
    df_pt.loc[df_pt['data'] == int(data), 'pdf'] = 1


In [25]:
df_pt_equalizado = df_pt.drop(df_pt[df_pt['data'].astype(int)<20000101].index)
print(df_pt_equalizado)

         data                                          texto_ata  pdf decisao
0    20250319  A) Atualização da conjuntura econômica e do ce...    0    None
1    20250129  A) Atualização da conjuntura econômica e do ce...    0    None
2    20241211  A) Atualização da conjuntura econômica e do ce...    0    None
3    20241106  A) Atualização da conjuntura econômica e do ce...    0    None
4    20240918  A) Atualização da conjuntura econômica e do ce...    0    None
..        ...                                                ...  ...     ...
222  20000524  Sumário Demanda e oferta agregadas Ambiente ex...    0    None
223  20000419  Sumário Demanda e oferta agregadas Ambiente ex...    0    None
224  20000322  Sumário Demanda e oferta agregadas Ambiente ex...    0    None
225  20000216  Sumário Demanda e oferta agregadas Ambiente ex...    0    None
226  20000119  Sumário Demanda e oferta agregadas Ambiente ex...    0    None

[227 rows x 4 columns]


In [26]:
df_pt_equalizado['data'].values


array([20250319, 20250129, 20241211, 20241106, 20240918, 20240731,
       20240619, 20240508, 20240320, 20240131, 20231213, 20231101,
       20230920, 20230802, 20230621, 20230503, 20230322, 20230201,
       20221207, 20221026, 20220921, 20220803, 20220615, 20220504,
       20220316, 20220202, 20211208, 20211027, 20210922, 20210804,
       20210616, 20210505, 20210317, 20210120, 20201209, 20201028,
       20200916, 20200805, 20200617, 20200506, 20200318, 20200205,
       20191211, 20191030, 20190918, 20190731, 20190619, 20190508,
       20190320, 20190206, 20181212, 20181031, 20180919, 20180801,
       20180620, 20180516, 20180321, 20180207, 20171206, 20171025,
       20170906, 20170726, 20170531, 20170412, 20170222, 20170111,
       20161130, 20161019, 20160831, 20160720, 20160608, 20160427,
       20160302, 20160120, 20151125, 20151021, 20150902, 20150729,
       20150603, 20150429, 20150304, 20150121, 20141203, 20141029,
       20140903, 20140716, 20140528, 20140402, 20140226, 20140

In [27]:
selic = [14.25, 13.15, 12.15, 11.15, 10.65, 10.40, 10.40, 10.40, 10.65, 11.15, 11.65, 12.15, 12.65, 13.15, 13.65, 13.65, 13.65, 13.65, 13.65, 13.65, 13.65, 13.65, 13.15, 12.65, 11.65, 10.65, 9.15, 7.65, 6.15, 5.15, 4.15, 3.4, 2.65, 1.90, 1.90, 1.90, 1.90, 1.90, 2.15, 2.90, 3.65, 4.15, 4.40, 4.90, 5.40, 5.90, 6.40, 6.40, 6.40, 6.40, 6.40, 6.40, 6.40, 6.40, 6.40, 6.40, 6.40, 6.65, 6.90, 7.40, 8.15, 9.15, 10.15, 11.15, 12.15, 12.90, 13.65, 13.90, 14.15, 14.15, 14.15, 14.15, 14.15, 14.15, 14.15, 14.15, 14.15, 14.15, 13.65, 13.15, 12.65, 12.15, 11.65, 11.15, 10.90, 10.90, 10.90, 10.90, 10.65, 10.40, 9.90, 9.40, 8.90, 8.40, 7.90, 7.40, 7.16, 7.12, 7.14, 7.14, 7.39, 7.89, 8.39, 8.90, 9.65, 10.4, 10.90, 11.40, 11.90, 12.42, 12.17, 11.92, 11.67, 11.17, 10.66, 10.66, 10.66, 10.66, 10.16, 9.40, 8.65, 8.65, 8.65, 8.65, 8.65, 8.65, 9.16, 10.16, 11.16, 12.66, 13.66, 13.65, 13.66, 12.92, 12.17, 11.63, 11.18, 11.18, 11.18, 11.18, 11.18, 11.43, 11.93, 12.43, 12.68, 12.93, 13.19, 13.67, 14.17, 14.67, 15.18, 15.70, 16.50, 17.26, 18.00, 18.49, 18.98, 19.48, 19.74, 19.75, 19.73, 19.75, 19.51, 19.24, 18.75, 18.25, 17.74, 17.23, 16.74, 16.23, 15.9, 15.83, 15.79, 15.79, 15.8, 16.09, 16.28, 16.30, 16.32, 17.32, 18.84, 19.84, 21.84, 24.32, 25.74, 26.27, 26.32, 26.32, 26.3, 25.36, 24.90, 21.90, 20.90, 20.90, 17.90, 17.87, 17.86, 18.40, 18.07, 18.35, 18.45, 18.80, 19.05, 19.05, 19.05, 19.05, 19.07, 19.04, 18.96, 18.31, 16.76, 16.30, 15.84, 15.2, 15.19, 15.76, 16.38, 16.56, 16.6, 16.54, 16.51, 17.35, 18.39, 18.55, 18.94, 18.88, 18.87]

In [28]:
decision = []
for i in range(len(selic)):
    if i == len(selic)-1:
        decision.append(-1)
    else:
        if selic[i] < selic[i+1]:
            decision.append(-1)
        elif selic[i] == selic[i+1]:
            decision.append(0)
        else:
            decision.append(1)

print(f"selic {selic}")
print(f"decisao {decision}")



selic [14.25, 13.15, 12.15, 11.15, 10.65, 10.4, 10.4, 10.4, 10.65, 11.15, 11.65, 12.15, 12.65, 13.15, 13.65, 13.65, 13.65, 13.65, 13.65, 13.65, 13.65, 13.65, 13.15, 12.65, 11.65, 10.65, 9.15, 7.65, 6.15, 5.15, 4.15, 3.4, 2.65, 1.9, 1.9, 1.9, 1.9, 1.9, 2.15, 2.9, 3.65, 4.15, 4.4, 4.9, 5.4, 5.9, 6.4, 6.4, 6.4, 6.4, 6.4, 6.4, 6.4, 6.4, 6.4, 6.4, 6.4, 6.65, 6.9, 7.4, 8.15, 9.15, 10.15, 11.15, 12.15, 12.9, 13.65, 13.9, 14.15, 14.15, 14.15, 14.15, 14.15, 14.15, 14.15, 14.15, 14.15, 14.15, 13.65, 13.15, 12.65, 12.15, 11.65, 11.15, 10.9, 10.9, 10.9, 10.9, 10.65, 10.4, 9.9, 9.4, 8.9, 8.4, 7.9, 7.4, 7.16, 7.12, 7.14, 7.14, 7.39, 7.89, 8.39, 8.9, 9.65, 10.4, 10.9, 11.4, 11.9, 12.42, 12.17, 11.92, 11.67, 11.17, 10.66, 10.66, 10.66, 10.66, 10.16, 9.4, 8.65, 8.65, 8.65, 8.65, 8.65, 8.65, 9.16, 10.16, 11.16, 12.66, 13.66, 13.65, 13.66, 12.92, 12.17, 11.63, 11.18, 11.18, 11.18, 11.18, 11.18, 11.43, 11.93, 12.43, 12.68, 12.93, 13.19, 13.67, 14.17, 14.67, 15.18, 15.7, 16.5, 17.26, 18.0, 18.49, 18.98, 19

In [29]:
df_pt_equalizado = df_pt_equalizado.sort_values(by='data', ascending=False)

In [30]:
df_pt_equalizado['decisao'] = decision
df_pt_equalizado['selic'] = selic

In [31]:
print(df_pt_equalizado)

         data                                          texto_ata  pdf  \
0    20250319  A) Atualização da conjuntura econômica e do ce...    0   
1    20250129  A) Atualização da conjuntura econômica e do ce...    0   
2    20241211  A) Atualização da conjuntura econômica e do ce...    0   
3    20241106  A) Atualização da conjuntura econômica e do ce...    0   
4    20240918  A) Atualização da conjuntura econômica e do ce...    0   
..        ...                                                ...  ...   
222  20000524  Sumário Demanda e oferta agregadas Ambiente ex...    0   
223  20000419  Sumário Demanda e oferta agregadas Ambiente ex...    0   
224  20000322  Sumário Demanda e oferta agregadas Ambiente ex...    0   
225  20000216  Sumário Demanda e oferta agregadas Ambiente ex...    0   
226  20000119  Sumário Demanda e oferta agregadas Ambiente ex...    0   

     decisao  selic  
0          1  14.25  
1          1  13.15  
2          1  12.15  
3          1  11.15  
4          1 

In [32]:
df_pt_equalizado.to_csv("../../dataset/pt/atas_dataset.csv", index=False, quoting=1, encoding="utf-8")